In [85]:
import os
import sys
module_path = os.path.abspath(os.path.join("..", "src"))
sys.path.insert(0, str(module_path))
from eduhub_queries import EdHubDB


In [86]:
# Part 1

edhub = EdHubDB()
edhub.build_collection()

Created collection: users
Created collection: courses
Created collection: lessons
Created collection: assignments
Created collection: enrollments
Created collection: submissions


In [87]:
# Part 2
edhub.seed_database()

Seeded 20 documents into 'users' collection.
Seeded 8 documents into 'courses' collection.
Seeded 15 documents into 'enrollments' collection.
Seeded 15 documents into 'lessons' collection.
Seeded 10 documents into 'assignments' collection.
Seeded 12 documents into 'submissions' collection.


## Part 3 - Basic CRUD Operations
###  Create Operations

In [88]:
# insert user

user_id = "u100"
edhub.insert_student(edhub.helper.make_user(user_id))

# verify user is inserted
edhub.users_col.find_one({"userId": user_id})

{'_id': ObjectId('684da2ca2dbbd81840fbad21'),
 'userId': 'u100',
 'email': 'framsey@example.org',
 'firstName': 'Steven',
 'lastName': 'Dixon',
 'role': 'student',
 'dateJoined': datetime.datetime(2024, 10, 22, 13, 22, 18, 568000),
 'profile': {'bio': 'It leave market like reality else participant.',
  'avatar': 'https://dummyimage.com/234x864',
  'skills': ['Machine Learning', 'Cloud Computing', 'Kubernetes']},
 'is_active': False}

In [89]:
# insert course
course_id = "c100"
user_id = "u2" # role is instructor
edhub.insert_course(edhub.helper.make_course(course_id=course_id, instructor_id=user_id))

# verify course is inserted
edhub.courses_col.find_one({"courseId": course_id})

{'_id': ObjectId('684da2ca2dbbd81840fbad22'),
 'courseId': 'c100',
 'title': 'Radio certainly art return human yard.',
 'description': 'Out increase claim already outside if. Land single need subject throw ball. Stuff minute fight act level.',
 'instructorId': 'u2',
 'category': 'JavaScript',
 'level': 'intermediate',
 'duration': 24,
 'price': 8051,
 'tags': ['MongoDB', 'Data Engineering'],
 'createdAt': datetime.datetime(2023, 4, 20, 5, 37, 18, 635000),
 'updatedAt': datetime.datetime(2025, 2, 13, 3, 57, 21, 36000),
 'isPublished': False}

In [90]:
# Enroll a student into a course
user_id = "u100" # existing user that's a student
course_id = "c100" # existing course

inserted_id = edhub.register_student(student_id=user_id, course_id=course_id)

# verify enrollment is created
print(inserted_id)
edhub.enrollments_col.find_one({"_id": inserted_id})

684da2ca2dbbd81840fbad23


{'_id': ObjectId('684da2ca2dbbd81840fbad23'),
 'enrollmentId': 'e16',
 'studentId': 'u100',
 'courseId': 'c100',
 'enrollmentDate': datetime.datetime(2012, 12, 12, 21, 13, 15, 199000),
 'progress': 0.0,
 'completed': False,
 'certificateIssued': False}

In [91]:
# Add a new lesson to an existing course
course_id = "c100" # existing course
lesson_id = "l100"

edhub.insert_lesson(edhub.helper.make_lesson(lesson_id=lesson_id, course_id=course_id))

# verify lesson is created
edhub.lessons_col.find_one({"lessonId": lesson_id})

{'_id': ObjectId('684da2ca2dbbd81840fbad24'),
 'lessonId': 'l100',
 'courseId': 'c100',
 'title': 'Hospital time while level me sometimes.',
 'content': 'Chance indicate most across unit. Performance evening provide sing. Read party impact. Policy kid what experience spend operation who. Sea increase yeah move.',
 'order': 69,
 'resources': ['intro.pdf'],
 'duration': 30,
 'createdAt': datetime.datetime(2022, 10, 30, 5, 25, 0, 477000),
 'updatedAt': datetime.datetime(2024, 7, 10, 21, 9, 10, 140000)}

### Read Operations

In [92]:
# Find all active students
edhub.get_active_students()

[{'_id': ObjectId('684da2ca2dbbd81840fbacd1'),
  'userId': 'u1',
  'email': 'alice.student@example.com',
  'firstName': 'Alice',
  'lastName': 'Johnson',
  'role': 'student',
  'dateJoined': datetime.datetime(2024, 1, 10, 9, 0),
  'profile': {'bio': 'Aspiring developer.',
   'avatar': 'alice.jpg',
   'skills': ['JavaScript', 'HTML']},
  'isActive': True},
 {'_id': ObjectId('684da2ca2dbbd81840fbacd3'),
  'userId': 'u3',
  'email': 'carol.student@example.com',
  'firstName': 'Carol',
  'lastName': 'Lee',
  'role': 'student',
  'dateJoined': datetime.datetime(2024, 2, 1, 11, 0),
  'profile': {'bio': 'Interested in data science.',
   'avatar': 'carol.jpg',
   'skills': ['Python', 'Pandas']},
  'isActive': True},
 {'_id': ObjectId('684da2ca2dbbd81840fbacd5'),
  'userId': 'u5',
  'email': 'eve.student@example.com',
  'firstName': 'Eve',
  'lastName': 'Davis',
  'role': 'student',
  'dateJoined': datetime.datetime(2024, 3, 5, 13, 0),
  'profile': {'bio': 'Frontend enthusiast.',
   'avatar': '

In [93]:
# Retrieve course details with instructor information
edhub.get_course_details()

[{'_id': ObjectId('684da2ca2dbbd81840fbace5'),
  'courseId': 'c1',
  'title': 'Introduction to JavaScript',
  'description': 'Learn the basics of JavaScript.',
  'instructorId': 'u2',
  'category': 'Programming',
  'level': 'beginner',
  'duration': 10,
  'price': 0,
  'tags': ['JavaScript', 'Web'],
  'createdAt': datetime.datetime(2024, 1, 1, 9, 0),
  'updatedAt': datetime.datetime(2024, 1, 10, 9, 0),
  'isPublished': False,
  'instructor': {'_id': ObjectId('684da2ca2dbbd81840fbacd2'),
   'userId': 'u2',
   'email': 'bob.instructor@example.com',
   'firstName': 'Bob',
   'lastName': 'Smith',
   'role': 'instructor',
   'dateJoined': datetime.datetime(2023, 12, 15, 10, 0),
   'profile': {'bio': 'Full-stack instructor.',
    'avatar': 'bob.jpg',
    'skills': ['Node.js', 'MongoDB']},
   'isActive': True}},
 {'_id': ObjectId('684da2ca2dbbd81840fbace6'),
  'courseId': 'c2',
  'title': 'Advanced Python',
  'description': 'Deep dive into Python.',
  'instructorId': 'u8',
  'category': 'Prog

In [94]:
# Get all courses in a specific category
edhub.get_courses_by_category("Programming")

[{'_id': ObjectId('684da2ca2dbbd81840fbace5'),
  'courseId': 'c1',
  'title': 'Introduction to JavaScript',
  'description': 'Learn the basics of JavaScript.',
  'instructorId': 'u2',
  'category': 'Programming',
  'level': 'beginner',
  'duration': 10,
  'price': 0,
  'tags': ['JavaScript', 'Web'],
  'createdAt': datetime.datetime(2024, 1, 1, 9, 0),
  'updatedAt': datetime.datetime(2024, 1, 10, 9, 0),
  'isPublished': False},
 {'_id': ObjectId('684da2ca2dbbd81840fbace6'),
  'courseId': 'c2',
  'title': 'Advanced Python',
  'description': 'Deep dive into Python.',
  'instructorId': 'u8',
  'category': 'Programming',
  'level': 'advanced',
  'duration': 20,
  'price': 50,
  'tags': ['Python', 'Advanced'],
  'createdAt': datetime.datetime(2024, 2, 1, 9, 0),
  'updatedAt': datetime.datetime(2024, 2, 10, 9, 0),
  'isPublished': True}]

In [95]:
# Find students enrolled in a particular course
course_id = "c1"
edhub.get_student_enrolled_to_course(course_id=course_id)

[{'_id': ObjectId('684da2ca2dbbd81840fbacd1'),
  'userId': 'u1',
  'email': 'alice.student@example.com',
  'firstName': 'Alice',
  'lastName': 'Johnson',
  'role': 'student',
  'dateJoined': datetime.datetime(2024, 1, 10, 9, 0),
  'profile': {'bio': 'Aspiring developer.',
   'avatar': 'alice.jpg',
   'skills': ['JavaScript', 'HTML']},
  'isActive': True},
 {'_id': ObjectId('684da2ca2dbbd81840fbace1'),
  'userId': 'u17',
  'email': 'quinn.student@example.com',
  'firstName': 'Quinn',
  'lastName': 'Lewis',
  'role': 'student',
  'dateJoined': datetime.datetime(2024, 6, 11, 9, 0),
  'profile': {'bio': 'Interested in DevOps.',
   'avatar': 'quinn.jpg',
   'skills': ['Docker', 'CI/CD']},
  'isActive': True}]

In [96]:
# Search courses by title (case-insensitive, partial match)
partial_title = "py"

edhub.search_courses_by_title(partial_title)

[{'_id': ObjectId('684da2ca2dbbd81840fbace6'),
  'courseId': 'c2',
  'title': 'Advanced Python',
  'description': 'Deep dive into Python.',
  'instructorId': 'u8',
  'category': 'Programming',
  'level': 'advanced',
  'duration': 20,
  'price': 50,
  'tags': ['Python', 'Advanced'],
  'createdAt': datetime.datetime(2024, 2, 1, 9, 0),
  'updatedAt': datetime.datetime(2024, 2, 10, 9, 0),
  'isPublished': True}]

### Update Operations

In [97]:
# Update a user’s profile information
user_id = "u1"

user = edhub.users_col.find_one({"userId": user_id})

print("Original: ", user["profile"])

profile = {
    "bio": "Software engineer",
    "avatar": "alice.png",
    "skills": ["Ruby", "Go"]
}

edhub.modify_profile(user_id=user_id, updates=profile)

user = edhub.users_col.find_one({"userId": user_id})

print("Updated: ", user["profile"])

Original:  {'bio': 'Aspiring developer.', 'avatar': 'alice.jpg', 'skills': ['JavaScript', 'HTML']}
Updated:  {'bio': 'Software engineer', 'avatar': 'alice.png', 'skills': ['Ruby', 'Go']}


In [ ]:
# Mark a course as published
course_id = "c1"

course = edhub.courses_col.find_one({"courseId": course_id})
print("Original: ", course["isPublished"])

edhub.publish_course(course_id=course_id)
course = edhub.courses_col.find_one({"courseId": course_id})
print("Updated: ", course["isPublished"])


Original:  False
Updated:  True


In [102]:
# Update assignment grade

submission_id = "s1"
grade = 40
feedback = "Good work"

submission = edhub.submissions_col.find_one({"submissionId": submission_id})
print("Original: ", submission["grade"])

edhub.update_assignment_grade(submission_id=submission_id, grade=grade, feedback=feedback)

submission = edhub.submissions_col.find_one({"submissionId": submission_id})
print("Updated: ", submission["grade"])

Original:  40
Updated:  40


In [103]:
# Add tags to an existing course
course_id = "c1"

course = edhub.courses_col.find_one({"courseId": course_id})
print("Original: ", course["tags"])

edhub.add_tags_to_course(course_id=course_id, tags=["Nodejs"])

course = edhub.courses_col.find_one({"courseId": course_id})
print("Updated: ", course["tags"])


Original:  ['JavaScript', 'Web']
Updated:  ['JavaScript', 'Web', 'Nodejs']


### Delete Operations